In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, f1_score, accuracy_score
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM
from sklearn.utils import class_weight

import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import EarlyStopping
from scipy.stats import pearsonr

from functions.nn import *

from fpdf import FPDF
import os

In [14]:
data = pd.read_csv('training_data/eurusd_final_dataset_2025.csv')
data = data.drop(['Date_Time'], axis=1)

X = data.drop(['label'], axis=1)
y = data['label'].astype(int)

In [ ]:
# Parameters
window_size = 5000
val_size = 1000
step = 1000

cost_per_trade = 1.5  # in pips

pip_value_per_standard_lot = 10 # Assuming EUR/USD and USD account (this is for a standard lot)
initial_account_balance = 10000.0
risk_per_trade_percentage = 0.01

close_prices = X['Close'].values
highs_full = X['High'].values
lows_full = X['Low'].values

# If any are 1-element tuples, extract the array
if isinstance(highs_full, tuple) and len(highs_full) == 1:
    highs_full = highs_full[0]
if isinstance(lows_full, tuple) and len(lows_full) == 1:
    lows_full = lows_full[0]
if isinstance(close_prices, tuple) and len(close_prices) == 1:
    close_prices = close_prices[0]

# Tracking
f1_per_window, acc_per_window, trade_per_window, profit_per_window = [], [], [], []
window_indices = []

current_account_balance_at_window_start = initial_account_balance

class_to_direction = {0: -1, 1: -1, 2: 0, 3: 1, 4: 1}

# Prediction horizon
steps = int(7)  # 7 candles
extra_steps = 0  # No extra steps for now

# window_length = 64

winning_trades = 0
losing_trades = 0

for i, start in enumerate(range(0, len(X) - window_size - val_size - steps, step)):

    # Standardize per window
    scaler = StandardScaler()
    scaler.fit(X[start : start + window_size])

    # Transform both the training data and the validation data
    X_scaled_train = scaler.transform(X[start : start + window_size])
    X_scaled_val = scaler.transform(X[start + window_size : start + window_size + val_size])
    train_X = X_scaled_train
    val_X = X_scaled_val

    val_y = y[start+window_size:start+window_size+val_size]
    val_y_cat = to_categorical(val_y, num_classes=5)

    train_y = y[start:start+window_size]
    train_y_cat = to_categorical(train_y, num_classes=5)

    input_features = train_X.shape[1]

    # train_X_seq, train_y_seq = create_lstm_sequences(train_X, train_y_cat, window_length)
    # val_X_seq, val_y_seq = create_lstm_sequences(val_X, val_y_cat, window_length)

    # 2. Optimize SL/TP on training window
    sl_tp_map = optimize_sl_tp_per_class(
        y=train_y,
        close_prices=close_prices,
        highs=highs_full,
        lows=lows_full,
        sl_values=[8, 10, 12, 15, 20],
        tp_values=[10, 12, 15, 20, 25],
        class_to_direction=class_to_direction,
        cost_per_trade=cost_per_trade
    )

    # 3. Estimate label horizon per class
    # Fui ver e isto varia bastante entre as windows por isso deixar
    avg_duration_by_class = estimate_avg_duration_per_class(
        y=train_y,
        close_prices=close_prices,
        highs=highs_full,
        lows=lows_full,
        sl_tp_map=sl_tp_map,
        class_to_direction=class_to_direction
    )

    '''# 4. Relabel training window using updated SL/TP and horizons
    train_y = relabel_data(
        X_window,
        sl_tp_map=sl_tp_map,
        avg_duration_by_class=avg_duration_by_class,
        class_to_direction=class_to_direction
    )
    train_y_cat = to_categorical(train_y, num_classes=5)'''

    cw = dict(enumerate(class_weight.compute_class_weight(
        class_weight='balanced', classes=np.unique(train_y), y=train_y)))

    model = build_model_nn(input_features)
    model.fit(train_X, train_y_cat,
              validation_data=(val_X, val_y_cat),
              epochs=30, batch_size=32,
              class_weight=cw,
              callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)],
              verbose=0)

    preds = np.argmax(model.predict(val_X, verbose=0), axis=1)
    # val_y_seq_labels = np.argmax(val_y, axis=1)

    f1 = f1_score(val_y, preds, average='weighted')
    acc = accuracy_score(val_y, preds)

    # Trade simulation
    val_start = start + window_size
    max_len = min(val_size, len(data) - val_start - steps)

    profit = 0.0
    trades = 0.0

    running_balance_in_window = current_account_balance_at_window_start

    # Fix tuple issue for close_prices and highs_full
    close_prices_arr = close_prices[0] if isinstance(close_prices, tuple) else close_prices
    highs_full_arr = highs_full[0] if isinstance(highs_full, tuple) else highs_full
    lows_full_arr = lows_full[0] if isinstance(lows_full, tuple) else lows_full

    # Use correct entry_prices and future_highs_seq
    entry_prices_arr = close_prices_arr[val_start:val_start + max_len]
    future_highs_seq_arr = [highs_full_arr[t:t+steps] for t in range(val_start, val_start + max_len)]
    future_lows_seq = [lows_full_arr[t:t+steps] for t in range(val_start, val_start + max_len)]

    for pred, entry, highs_seq, lows_seq in zip(preds[:max_len], entry_prices_arr, future_highs_seq_arr, future_lows_seq):
        direction = class_to_direction.get(pred, 0)
        if direction == 0:
            continue

        sltp = sl_tp_map.get(pred, {'sl': None, 'tp': None})
        if sltp['sl'] is None or sltp['tp'] is None:
            continue

        current_sl_pips = sltp['sl'] # ADDED: Get SL for lot size calculation
        current_tp_pips = sltp['tp'] # ADDED: Get TP for clarity

        # ADDED: Ensure SL is valid for lot size calculation
        if current_sl_pips is None or current_sl_pips <= 0:
            print(f"Warning: SL for pred {pred} is {current_sl_pips}. Skipping trade due to invalid SL for lot size calculation.")
            continue

        # Calculate monetary risk for this specific trade
        monetary_risk_for_this_trade = running_balance_in_window * risk_per_trade_percentage

        # Calculate the lot size multiplier required for this trade
        calculated_lot_size_multiplier = monetary_risk_for_this_trade / (current_sl_pips * pip_value_per_standard_lot)

        # Apply broker's minimum and maximum lot size constraints
        min_broker_lot_size = 0.01 # Example: Minimum micro lot
        max_broker_lot_size = 50.0 # Example: Maximum standard lots allowed
        

        calculated_lot_size_multiplier = min(max_broker_lot_size, calculated_lot_size_multiplier)
        calculated_lot_size_multiplier = round(calculated_lot_size_multiplier, 2)
        if calculated_lot_size_multiplier < min_broker_lot_size:
            continue
        
        '''# Use a fixed lot size per trade (e.g., 1 standard lot)
        fixed_lot_size = 1.0  # You can try other values like 0.1 or 0.5

        # Skip trades that fall outside broker limits
        if fixed_lot_size < min_broker_lot_size or fixed_lot_size > max_broker_lot_size:
            continue

        calculated_lot_size_multiplier = fixed_lot_size'''

        # Dynamic candle limit per class
        limit = avg_duration_by_class.get(pred) + extra_steps
        highs_limited = highs_seq[:limit]
        lows_limited = lows_seq[:limit]

        result, _ = simulate_trade(entry, highs_limited, lows_limited, direction, sltp['sl'], sltp['tp'])
        result -= cost_per_trade
        trades += 1

        # Count win/loss
        if result > 0:
            winning_trades += 1
        elif result < 0:
            losing_trades += 1

        trade_monetary_profit = result * pip_value_per_standard_lot * calculated_lot_size_multiplier
        profit += trade_monetary_profit
        running_balance_in_window += trade_monetary_profit

    # Log
    trade_per_window.append(trades)
    profit_per_window.append(profit)
    f1_per_window.append(f1)
    acc_per_window.append(acc)
    window_indices.append(i)

    current_account_balance_at_window_start = running_balance_in_window

    print(f"Window {i}: F1 = {f1:.3f}, Accuracy = {acc:.3f}, Profit = {profit:.2f}, Trades = {trades}, Current Balance: {current_account_balance_at_window_start:.2f}")

Window 0: F1 = 0.515, Accuracy = 0.476, Profit = 3175.06, Trades = 578.0, Current Balance: 13175.06
Window 1: F1 = 0.393, Accuracy = 0.378, Profit = 10391.25, Trades = 647.0, Current Balance: 23566.31
Window 2: F1 = 0.603, Accuracy = 0.559, Profit = 3669.30, Trades = 415.0, Current Balance: 27235.61
Window 3: F1 = 0.604, Accuracy = 0.561, Profit = 11317.17, Trades = 446.0, Current Balance: 38552.78
Window 4: F1 = 0.579, Accuracy = 0.517, Profit = 6869.57, Trades = 513.0, Current Balance: 45422.35
Window 5: F1 = 0.629, Accuracy = 0.592, Profit = 7134.97, Trades = 377.0, Current Balance: 52557.32
Window 6: F1 = 0.562, Accuracy = 0.510, Profit = 341.52, Trades = 471.0, Current Balance: 52898.84
Window 7: F1 = 0.344, Accuracy = 0.355, Profit = -8408.23, Trades = 574.0, Current Balance: 44490.61
Window 8: F1 = 0.239, Accuracy = 0.254, Profit = 31369.15, Trades = 807.0, Current Balance: 75859.76
Window 9: F1 = 0.513, Accuracy = 0.454, Profit = 25504.35, Trades = 602.0, Current Balance: 10136

In [16]:
generate_model_report_pdf(
    steps,
    extra_steps,
    window_indices,
    f1_per_window,
    acc_per_window,
    profit_per_window, # Ensure this list contains monetary profits ($)
    trade_per_window,           # Ensure this list contains total trades for each window
    initial_account_balance,
    # Parameters
    window_size,
    val_size,
    step,
    cost_per_trade,
    pip_value_per_standard_lot, # Corrected name for clarity
    risk_per_trade_percentage,        # Corrected name for clarity (e.g., 0.1 for mini lot)
    winning_trades,
    losing_trades,
    report_filename="model_timedata_risk_precentage_2025.pdf"
)


Report generated successfully: model_timedata_risk_precentage_2025.pdf
